In [1]:
import pandas as pd
from pypfopt.expected_returns import returns_from_prices 
from data_capture import FromYF
import inputs
from core.regression import regression
from core.logic import Filtration, Allocation
from core.performance import regressed_expected_returns

In [2]:
watchlist_df = pd.read_excel(inputs.WATCHLIST_PATH,index_col=0)
watchlist = watchlist_df.index.tolist()
watchlist.append(inputs.MARKET)

price_df = FromYF.get_prices(watchlist,period=inputs.PERIOD,interval=inputs.INTERVAL)

returns_df = returns_from_prices(price_df,log_returns=inputs.IS_CONTINOUSLY_COMPOUNDED)

returns_df

[*********************100%***********************]  12 of 12 completed


,BLX,CSIQ,CWEN,ED,ENPH,FSLR,GPRE,MAXN,ORA,REX,SEDG,^RUA
Date,,,,,,,,,,,,
2021-03-11,0.003015,0.083795,0.011299,-0.007816,0.084695,0.069995,0.030815,0.130239,0.018804,-0.005472,0.055637,0.012878
2021-03-12,-0.004209,-0.020660,-0.002793,0.016041,-0.003810,0.011479,0.009843,-0.007947,0.002293,-0.035371,-0.012702,0.001374
2021-03-15,0.008454,-0.024793,0.009454,0.012405,0.057374,0.007078,-0.018051,0.068091,-0.037058,-0.035944,0.019658,0.007085
2021-03-16,-0.009581,-0.051740,-0.017690,0.003620,-0.045074,-0.000848,-0.044118,-0.038000,-0.023043,-0.064425,-0.039063,-0.003661
2021-03-17,0.001209,-0.026341,-0.048729,-0.004023,-0.014972,-0.028135,0.019231,-0.038202,-0.033678,0.018069,0.020413,0.003177
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-03,0.000000,0.021032,0.022911,0.012052,0.026470,0.059984,0.020444,0.045810,0.009024,0.020796,0.010543,0.016014
2023-03-06,0.016199,0.022994,-0.008833,0.010365,0.037714,0.010423,-0.041786,0.022436,0.012383,-0.047535,0.017287,-0.001377
2023-03-07,0.013815,-0.003512,-0.004774,-0.010586,-0.005059,0.009373,-0.014038,-0.020376,-0.008959,-0.000930,-0.012362,-0.014719


In [3]:
regression_df = regression(returns_df, inputs.MARKET)

selection_df = Filtration.significance_by_beta(regression_df,inputs.MIN_BETA, inputs.MAX_BETA)

selection_df

,alpha,betav,alpha_t_stat,beta_t_stat,skewness,kurtoses,betaub,betalb,alphalb,alphaub,epsilon
tickers,,,,,,,,,,,
BLX,0.000497,0.556598,0.801715,11.328550,0.791,7.571,0.653130,0.460066,-0.000721,0.001715,0.157163
CSIQ,0.000598,1.239148,0.390031,10.195359,0.622,4.717,1.477941,1.000354,-0.002414,0.003610,0.349891
CWEN,0.000471,0.783577,0.700380,14.687746,-0.065,3.937,0.888394,0.678761,-0.000851,0.001794,0.221254
ED,0.000737,0.352312,1.443011,8.702991,-0.089,3.689,0.431848,0.272776,-0.000266,0.001740,0.099480
ENPH,0.001548,1.618803,0.941723,12.423941,0.849,8.505,1.874802,1.362805,-0.001682,0.004777,0.457092
FSLR,0.002509,1.012828,1.997934,10.173296,0.498,7.418,1.208431,0.817224,0.000042,0.004977,0.285986
GPRE,0.001107,1.431163,0.738604,12.046816,0.309,4.037,1.664573,1.197753,-0.001838,0.004051,0.404109
MAXN,0.001361,1.775102,0.538312,8.858428,1.244,11.009,2.168805,1.381398,-0.003606,0.006327,0.501225
ORA,0.000305,0.848341,0.342081,11.985978,-0.046,7.689,0.987400,0.709282,-0.001449,0.002060,0.239541


In [4]:
E_df = regressed_expected_returns(returns_df,inputs.MARKET)

E_df = E_df.loc[selection_df.index.values]

E_df

,predict,predict_lower,predict_upper
tickers,,,
BLX,0.000503,-0.026781,0.027787
CSIQ,0.000612,-0.066882,0.068106
CWEN,0.000480,-0.029146,0.030106
ED,0.000741,-0.021740,0.023221
ENPH,0.001566,-0.070791,0.073923
FSLR,0.002521,-0.052766,0.057807
GPRE,0.001123,-0.064849,0.067095
MAXN,0.001381,-0.109898,0.112659
ORA,0.000315,-0.038990,0.039619


In [5]:
cov_matrix_df = returns_df[selection_df.index.values].cov(min_periods=None, ddof=1)

cov_matrix_df

,BLX,CSIQ,CWEN,ED,ENPH,FSLR,GPRE,MAXN,ORA,REX,SEDG
BLX,0.000241,0.000112,0.000097,0.000047,0.000080,0.000105,0.000160,0.000090,0.000085,0.000129,0.000125
CSIQ,0.000112,0.001420,0.000263,0.000016,0.001052,0.000785,0.000564,0.001415,0.000382,0.000319,0.001032
CWEN,0.000097,0.000263,0.000324,0.000090,0.000328,0.000236,0.000182,0.000434,0.000200,0.000141,0.000320
ED,0.000047,0.000016,0.000090,0.000150,0.000014,0.000033,0.000009,0.000041,0.000068,0.000024,0.000040
ENPH,0.000080,0.001052,0.000328,0.000014,0.001769,0.000816,0.000568,0.001519,0.000438,0.000339,0.001366
FSLR,0.000105,0.000785,0.000236,0.000033,0.000816,0.000952,0.000423,0.001156,0.000313,0.000260,0.000804
GPRE,0.000160,0.000564,0.000182,0.000009,0.000568,0.000423,0.001450,0.000791,0.000306,0.000675,0.000609
MAXN,0.000090,0.001415,0.000434,0.000041,0.001519,0.001156,0.000791,0.003698,0.000566,0.000481,0.001502
ORA,0.000085,0.000382,0.000200,0.000068,0.000438,0.000313,0.000306,0.000566,0.000513,0.000198,0.000408
REX,0.000129,0.000319,0.000141,0.000024,0.000339,0.000260,0.000675,0.000481,0.000198,0.001031,0.000348


In [6]:
market_returns_s = returns_df[inputs.MARKET]

position_df = Allocation.by_method(E_df,cov_matrix_df,market_returns_s,inputs.ALLOCATION_METHOD)

position_df.to_csv(path_or_buf = inputs.POSITION_PATH)

position_df


,weights
BLX,0.000000
CSIQ,0.000000
CWEN,0.000000
ED,0.314528
ENPH,0.000000
FSLR,0.685472
GPRE,0.000000
MAXN,0.000000
ORA,0.000000
REX,0.000000
